In [1]:
import numpy as np
from time import time
t0 = time()

## Parameters

In [2]:
S = input("Stock price: ")
X = input("Strike price: ")
T = input("Matuirty in years: ")
s = input("Annual volatility: ")
r = input("Annual interest rate: ")
n = input("Number of periods: ")
k = input("Number of simulation paths: ")
dt = float(T) / n
R = np.exp(r*dt)

Stock price: 100
Strike price: 100
Matuirty in years: 1
Annual volatility: 0.3
Annual interest rate: 0.05
Number of periods: 100
Number of simulation paths: 100000


In [3]:
# S = 100
# X = 100
# T = 1
# s = 0.3
# r = 0.05
# n = 100
# k = 100000
# dt = float(T) / n
# R = np.exp(r*dt)

## Create Stock price table & Exercise value table & Cash flow table
$S_{i+1} = S_i e^{(r-\sigma^2/2)\Delta t +\sigma\sqrt{\Delta t}\space\xi}$

In [4]:
# Stock price table
P = np.zeros((k, n+1))
P[:, 0] = S
for j in range(1, n+1):
    power = s * np.sqrt(dt) * np.random.standard_normal(k) + (r-s*s/2) * dt
    P[:, j] = P[:, j-1] * np.exp(power)

total = np.zeros(k)
for j in range(n+1):
    total += P[:, j]
    P[:, j] = total / (j+1)
    
# Exercise value table
E = np.where(P < X, X - P, 0)

## Create cash flow table (Backward induction)

In [5]:
# Cash flow table
CF = E[:, n]

for j in range(n-1, 0, -1):
    # Discount
    CF /= R
    
    # Continuation value
    path = E[:, j] > 0
    x = P[path, j]
    if len(x) == 0:
        continue
    y = CF[path]
    coefficient = np.polyfit(x, y, 2) # Least-Square Sum
    Continue = np.polyval(coefficient, x)
    
    # Exercise value
    Exercise = E[path, j]
    
    # Consider early exercise
    CF[path] = np.where(Exercise > Continue, Exercise, CF[path])

## Year 0
$\text{estimated standard error }(ESE) = \text{sample standard deviation }(SD) / \sqrt{\text{number of paths }(k)}$

In [6]:
CF /= R
V = np.sum(CF) / k
if V < X - S:
    V = X - S
sd = np.std(CF) / np.sqrt(k)
print "Price: %.4f" % V
print "Standard error: %.4f" % sd 
print "User time: %.4f" % (time()-t0)

Price: 5.5675
Standard error: 0.0252
User time: 9.3165


In [7]:
# Assume S = 100, X = 100, T = 1, r = 0.05, s = 0.30, n = 100, and k = 100000. 
# The put price is 5.5126 and its standard error is 0.0251.